Começamos por Carregar as bibliotecas utilizadas neste Projeto

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import random
import seaborn as sns
import fsspec
import numpy as np
import seaborn as sns

Nossa base de dados

In [ ]:

base_dados = pd.read_csv('C:\Users\avsan\.vscode\Projeto Aplicado II\alt_fuel_stations (Mar 18 2024).csv')
base_dados.head()
base_dados.info()
base_dados.dropna()


Preparaçao e Tratamento dos dados

In [ ]:

base_dados.isnull().sum() / len(base_dados)

print('Before', len(base_dados))
base_dados = base_dados.dropna()
print('After', len(base_dados))

Analisando o Banco de Dados selecionamos as colunas para o projeto

In [ ]:

dados_selecionados = pd.DataFrame(base_dados)
colunas_selecionadas = ['City', 'State', 'Model Year', 'Make', 'Model',
                        'Electric Vehicle Type', 'Electric Range',
                        'Electric Utility', 'Vehicle Location'] 


df = dados_selecionados[colunas_selecionadas]

df.head(15)
df.tail()
df.index
df.columns
df.shape


Alteraçoes necessárias para proseguir com a Analise Exploratória

In [ ]:
df.rename(columns={'Electric Range': 'Electric_Range'}, inplace=True)
df.info


Agrupamos os dados por marca, separamos armazenamos em subdataframes e criamos um script para consulta

In [ ]:

# Separar os dados por marcas
marcas = df['Make'].unique()

# Dicionário para armazenar os subdataframes
subdataframes = {}

# Iterando sobre as marcas únicas
for marca in marcas:
    # Criando o subdataframe para cada marca
    subdataframes[marca] = df[df['Make'] == marca]

# Consultando e criando um novo dataset com as primeiras linhas do subdataframe por marca
for marca_desejada in subdataframes.keys():
    print(f"\nSubdataframe da marca {marca_desejada}:")
    sub_df = subdataframes[marca_desejada] #aqui pode colocar a consulta, ex .head()
    globals()[f"sub_df_{marca_desejada}"] = sub_df
    print(sub_df)



geramos graficos automatizados para todos os subdataframes para analise exploratoria.
Vamos contar a frequencia dos modelos de carros de cada marca


In [ ]:
# Defina o número de gráficos por página
graficos_por_pagina = 6
linhas_por_pagina = 2
colunas_por_pagina = 3

# Tamanho da fonte do título do gráfico
tamanho_fonte_titulo = 8

# Inicialize uma variável para contar os gráficos
contador_graficos = 0

# Inicialize uma lista vazia para armazenar os eixos
eixos = None

# Iterando sobre os subdatasets para gerar gráficos
for marca, sub_df in subdataframes.items():
    # Contando a frequência dos modelos de carros em cada marca
    contagem_modelos = sub_df['Model'].value_counts()
    
    # Verificando se é necessário criar uma nova página
    if contador_graficos % graficos_por_pagina == 0:
        # Criando uma nova figura e eixos
        fig, eixos = plt.subplots(linhas_por_pagina, colunas_por_pagina, figsize=(15, 10))
    
    # Calculando a posição do gráfico na página
    linha = contador_graficos // colunas_por_pagina % linhas_por_pagina
    coluna = contador_graficos % colunas_por_pagina
    
    # Criando o gráfico de barras no subplot correspondente
    contagem_modelos.plot(kind='bar', color='#1f77b4', edgecolor='black', linewidth=3, ax=eixos[linha, coluna])
    eixos[linha, coluna].set_title(f'Frequência dos modelos de carros da marca {marca}', fontsize=tamanho_fonte_titulo)
    eixos[linha, coluna].set_xlabel('Modelo')
    eixos[linha, coluna].set_ylabel('Frequência')
    eixos[linha, coluna].tick_params(axis='x', rotation=45)
    
    # Incrementando o contador de gráficos
    contador_graficos += 1

# Ajustando o layout da última página de gráficos
plt.tight_layout()
# Exibindo a última página de gráficos
plt.show()

Visualisando os dados entendemos ser necessária a separação entre carros eletricos 'BEV' e Hibridos 'PHEV'

# Significado das siglas BEV e PHEV
#BEV(Battery Electric Vehicle): Um BEV é um veiculo elétrico que é alimentado
#exclusivamente por bateria. Isso significa que ele nao possui
#um motor de combustão interna e depende apenas de uma bateria
#recarregável para fornecer energia.

#PHEV(Plug-in-Hybrid Electric Vehicle): Um PHEV é um tipo de
#veiculo eletrico que possui tanto um motor eletrico quanto
#um motor a combustão interna. Esses veiculos sao equipados
#com uma bateria recarregável que alimenta o motor elétrico
#e tambem possuem um tanque de combustivel para alimentar o motor
#a combustao

In [ ]:
# Defina o número de gráficos por página
graficos_por_pagina = 6

# Tamanho da fonte do título do gráfico
tamanho_fonte_titulo = 8

# Calculando o número de linhas e colunas para os gráficos
num_linhas, num_colunas = 2, 3
graf_por_pagina_por_area = graficos_por_pagina // (num_linhas * num_colunas)

# Inicialize uma variável para contar os gráficos
contador_graficos = 0

# Inicialize uma variável para contar as páginas
contador_paginas = 0

# Criando uma nova figura e eixos fora do loop de iteração
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(15, 10))

# Iterando sobre os subdatasets para contar o número de BEVs e PHEVs e gerar gráficos
for marca, sub_df in subdataframes.items():
    # Contando o número de BEVs e PHEVs em cada marca
    contagem_ev_type = sub_df['Electric Vehicle Type'].value_counts()
    
    # Criando o gráfico de barras
    linha = contador_graficos % num_linhas
    coluna = contador_graficos % num_colunas
    ax = axs[linha, coluna]
    contagem_ev_type.plot(kind='bar', color=['#1f77b4', '#ff7f0e'], edgecolor='black', linewidth=3, ax=ax)
    ax.set_title(f'Contagem de Electric Vehicle Type para a marca {marca}', fontsize=tamanho_fonte_titulo)
    ax.set_xlabel('Tipo de Veículo Elétrico')
    ax.set_ylabel('Contagem')
    ax.tick_params(axis='x', rotation=0)
    
    # Incrementando o contador de gráficos
    contador_graficos += 1

    # Verificando se é necessário criar uma nova página
    if contador_graficos % graf_por_pagina_por_area == 0:
        # Exibindo a página atual de gráficos
        plt.show()
        # Incrementando o contador de páginas
        contador_paginas += 1
        # Criando uma nova figura e eixos para a próxima página
        fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(15, 10))

# Ajustando o layout da última página de gráficos
plt.tight_layout()
# Exibindo a última página de gráficos
plt.show()

Agora sabemos que o foco do nosso projeto será os 'BEV' - Veiculos Elétricos, encontramos nosso Target.

In [ ]:
import matplotlib.pyplot as plt

# Defina o número de gráficos por página
graficos_por_pagina = 6

# Tamanho da fonte do título do gráfico
tamanho_fonte_titulo = 8

# Dicionário para armazenar o número de veículos BEV por marca
num_bev_por_marca = {}

# Iterar sobre os subdatasets originais
for marca, sub_df in subdataframes.items():
    # Filtrar o subdataset para selecionar apenas os carros do tipo BEV
    sub_df_bev = sub_df[sub_df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']

    # Armazenar o número de veículos BEV por marca
    num_bev = len(sub_df_bev)

    # Adicionar a marca e o número de veículos ao dicionário
    num_bev_por_marca[marca] = num_bev

# Ordenar o dicionário num_bev_por_marca em ordem decrescente por valor
num_bev_por_marca_ordenado = sorted(num_bev_por_marca.items(), key=lambda x: x[1], reverse=True)

# Cores para as barras
cores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Dividir o dicionário ordenado em lotes de tamanho graficos_por_pagina
lotes = [num_bev_por_marca_ordenado[i:i + graficos_por_pagina] for i in range(0, len(num_bev_por_marca_ordenado), graficos_por_pagina)]

# Criar e exibir os gráficos
for lote in lotes:
    fig, ax = plt.subplots(figsize=(10, 6))
    for i, (marca, num_bev) in enumerate(lote):
        ax.bar(marca, num_bev, color=cores[i], edgecolor='black', linewidth=3)
    ax.set_title('Número de veículos BEV por marca', fontsize=tamanho_fonte_titulo)
    ax.set_xlabel('Marca')
    ax.set_ylabel('Número de veículos BEV')
    ax.set_xticklabels([marca for marca, _ in lote], rotation=45)
    plt.tight_layout()
    plt.show()

Separamos nosso Target

In [ ]:
# Filtrar o dataset original para selecionar apenas os carros do tipo BEV
df_bev = df.loc[df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']

soma_quantidade_total = df_bev['Model'].value_counts()
total = soma_quantidade_total.sum()
print(total)
def consultar_veiculos_bev(df, marcas_de_interesse=None, modelos_de_interesse=None):
   
    print("Tamanho do DataFrame original:", len(df))  # Depuração: Verificar o tamanho do DataFrame original

    # Filtrar o dataset original para selecionar apenas os carros do tipo BEV
    df_bev = df[df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']

    print("Tamanho do DataFrame filtrado (BEV):", len(df_bev))  # Depuração: Verificar o tamanho do DataFrame filtrado

    # Filtrar por marcas de interesse (se especificado)
    if marcas_de_interesse:
        df_bev = df_bev[df_bev['Make'].isin(marcas_de_interesse)]

    print("Tamanho do DataFrame filtrado (marcas de interesse):", len(df_bev))  # Depuração: Verificar o tamanho do DataFrame filtrado

    # Filtrar por modelos de interesse (se especificado)
    if modelos_de_interesse:
        df_bev = df_bev[df_bev['Model'].isin(modelos_de_interesse)]

    print("Tamanho do DataFrame filtrado (modelos de interesse):", len(df_bev))  # Depuração: Verificar o tamanho do DataFrame filtrado

    # Agrupar os veículos BEV por marca e modelo, contar a quantidade e manter as colunas de interesse
    df_bev_por_marca_modelo = df_bev.groupby(['Make', 'Model']).agg(
        Quantidade=('Model', 'size'),  # Conta o número de linhas para cada grupo (ou seja, a quantidade de veículos)
        Electric_Range=('Electric_Range', 'first'),  # Pega a autonomia elétrica do primeiro veículo do grupo
        Vehicle_Location=('Vehicle Location', 'first')  # Pega a localização do primeiro veículo do grupo
    ).reset_index()

    print("Tamanho do DataFrame após a agregação:", len(df_bev_por_marca_modelo))  # Depuração: Verificar o tamanho do DataFrame após a agregação

    # Reordenar as colunas
    df_bev_por_marca_modelo_ordenado = df_bev_por_marca_modelo.sort_values(by='Quantidade', ascending=False)

    return df_bev_por_marca_modelo_ordenado


Importamos um novo Banco de Dados com Informações dos Pontos de Carregamento, para analisarmos a necessidade de aumento dos pontos

In [ ]:

df = pd.read_csv('D:/OneDrive - Instituto Presbiteriano Mackenzie/Mackenzie/Aulas/3 semestre/Projeto Aplicado II/alt_fuel_stations (Mar 18 2024).csv')


Selecionamos e trabalhamos as colunas que queriamos extrair as informaçoes

In [ ]:

colunas_selecionadas = df.copy()

colunas_selecionadas = colunas_selecionadas[['State','Groups With Access Code','EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'Latitude', 'Longitude']]


# Crie um novo DataFrame contendo apenas as linhas onde a coluna 'category' é igual a 'public'
colunas_selecionadas_public = colunas_selecionadas[colunas_selecionadas['Groups With Access Code'] == 'Public']


# Crie um novo DataFrame contendo apenas as linhas onde a coluna 'State' é igual a 'WA'
colunas_selecionadas_public_wa = colunas_selecionadas_public[colunas_selecionadas_public['State'] == 'WA']

# Crie uma cópia do DataFrame original para manter as outras colunas inalteradas
df_filtered = colunas_selecionadas_public_wa.copy()

# Selecione apenas as colunas específicas para verificar se todas estão preenchidas com NaN
columns_to_check = ['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']

# Remova as linhas onde todas as três colunas especificadas estão preenchidas com NaN
df_filtered = df_filtered.dropna(subset=columns_to_check, how='all')


df_filtered[['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']] = df_filtered[['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']].fillna(0)

colunas_selecionadas_preenchidas = df_filtered.copy()


Criamos graficos para analise exploratoria e entender como estava a distribuicao por capacidade dos pontos de recarga

In [ ]:

# Criando o gráfico de barras coloridas
plt.figure(figsize=(10, 6))
soma_instancias.plot(kind='bar', color=['blue', 'green', 'red'])
plt.title('Soma das Instâncias dos Pontos de Recarga por Capacidade')
plt.xlabel('Colunas')
plt.ylabel('Soma das Instâncias')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


colunas_selecionadas = ['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']
soma_total_pontos = colunas_selecionadas_public_wa[colunas_selecionadas].sum()
print(soma_total_pontos)
soma_total_pontos1 = 17+4175+1045
print(soma_total_pontos1)
soma_total = valores['EV Level1 EVSE Num'] + valores ['EV Level2 EVSE Num'] + valores['EV DC Fast Count']
print(soma_total)

# Exibindo a tabela com a soma das instâncias
print("\nTabela com a Soma das Instâncias das Colunas Selecionadas:")
print(soma_instancias)


selecionamos as coordenadas GPS Para plotarmos em um grafico, dos dois banco de dados

In [ ]:

colunas_gps = colunas_selecionadas_preenchidas[['Latitude', 'Longitude']]


In [ ]:
import pandas as pd
import re

# Função para extrair latitude e longitude de uma string no formato POINT (longitude latitude)
def extrair_latitude_longitude(point_string):
    # Use uma expressão regular para extrair os valores numéricos
    matches = re.findall(r"-?\d+\.\d+", point_string)
    if len(matches) >= 2:
        return float(matches[1]), float(matches[0])  # A ordem é invertida para corresponder ao formato (latitude, longitude)
    else:
        return None, None

seu_dataframe = base_dados
# Iterar sobre as linhas do dataframe e extrair latitude e longitude
dados_selecionados = []
for index, linha in seu_dataframe.iterrows():
    localizacao_str = linha['Vehicle Location']
    latitude, longitude = extrair_latitude_longitude(localizacao_str)
    if latitude is not None and longitude is not None:
        dados_selecionados.append({'Latitude': latitude, 'Longitude': longitude})

# Imprimir os dados
for dado in dados_selecionados:
    print(dado)
    
gps_selecionado_carros = dado

In [6]:
import webbrowser

# Caminho para o arquivo HTML
caminho_arquivo_html = r'D:\OneDrive - Instituto Presbiteriano Mackenzie\GitHub\google earth\imagem de distribuicao automoveis x pontos de carga.html'

# Abrir o arquivo HTML em um navegador padrão
webbrowser.open('file://' + caminho_arquivo_html)

True

No grafico conseguimos visualizar a distribuição dos automoveis eletricos e dos pontos de recarga, mas precisamos verificar isto em numeros.
Para isto realizmos calculos de densidade

In [ ]:
#Precisamos calcular a densidade de veiculos por unidade
#por area: Formula Densidade = Numero de veiculos/Area da Regiao em quilometros quadrados
#area da regiao de Washingtom em km2 177 quilometros segundo site https://www.greelane.com/pt/humanidades/geografia/washington-dc-geography-1435747/#:~:text=DC%20tem%2068%20milhas%20quadradas%20A%20%C3%A1rea%20total,m%29%20e%20est%C3%A1%20localizado%20no%20bairro%20de%20Tenleytown.
Formula_Densidade = total / 177
print(Formula_Densidade)


In [ ]:
#Densidade de Veiculos por ponto de recarga
#Formula Densidade = Numero de veiculos eletricos / Numero pontos de recarga
densidade_ponto_recarga = total / soma_total_pontos1
print(densidade_ponto_recarga)


In [ ]:
#Densidade de veiculos em relação a capacidade de carga.
#levar em consideração a capacidade de carga dos pontos em relação
#aos diferentes tipos de pontos com capacidade variada
#formula densidade = numero de veiculos / capacidade total dos pontos de recarga
capacidade_EV_Level1 = total / 17
print(capacidade_EV_Level1)


In [ ]:
capacidade_EV_Level2= total / 1045
print(capacidade_EV_Level1)



EV_DC_Fast_Count = total / 4175
print(capacidade_EV_Level1)


E analisar tendencias futuras usando graficos e estatisticas para analisar tendencias

In [ ]:
base_dados = pd.read_csv('D:/OneDrive - Instituto Presbiteriano Mackenzie/Mackenzie/Aulas/3 semestre/Projeto Aplicado II/Electric_Vehicle_Population_Data.csv', index_col=False)

print(base_dados.dtypes)

base = base_dados[base_dados['Model Year'] >= 2015]

base = base_dados[base_dados['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']


base_selecionada = base[['Model','Model Year', 'Electric Vehicle Type']]

base_selecionada = base_selecionada.groupby(["Model", "Model Year"]).size().reset_index(name="Quantidade")


# Exibindo o DataFrame com as instâncias relacionadas com 'Battery Electric Vehicle (BEV)'
print(base_selecionada)

base_selecionada_progressao = base_selecionada[['Model Year', 'Quantidade']]


#Definir a semente

random.seed(1234)
# Pré-processamento de dados
base_selecionada_progressao["Model Year"] = pd.to_numeric(base_selecionada_progressao["Model Year"])
base_selecionada_progressao.dropna(inplace=True)

# Selecionar variáveis
X = base_selecionada_progressao["Model Year"]
y = base_selecionada_progressao["Quantidade"]

# Dividir dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Treinar o modelo
model = LinearRegression()
model.fit(X_train.values.reshape(-1, 1), y_train)

# Avaliar o modelo
print("R²:", model.score(X_test.values.reshape(-1, 1), y_test))
print("RMSE:", np.sqrt(mean_squared_error(y_test, model.predict(X_test.values.reshape(-1, 1)))))

# Fazer previsões para um futuro "Model Year"
ano_futuro = 2030
previsao = model.predict([[ano_futuro]])

print(f"Previsão de vendas para {ano_futuro}: {previsao}")

# Criar um vetor de "Model Year" para as previsões
anos_futuro = np.arange(2023, 2030)

# Fazer previsões para os "Model Years" futuros
previsoes = model.predict(anos_futuro.reshape(-1, 1))

# Plotar os dados
plt.scatter(X_test, y_test, color="blue", label="Dados")
plt.plot(X_test, model.predict(X_test.values.reshape(-1, 1)), color="red", label="Linha de regressão")
plt.plot(anos_futuro, previsoes, color="green", label="Previsões")

# Ajustar o gráfico
plt.legend()
plt.xlabel("Model Year")
plt.ylabel("Quantidade")
plt.title("Regressão Linear")
plt.show()



Agora com base em suposições sobre o crescimento futuro


In [ ]:
# Cenário otimista: crescimento de 10% ao ano
optimistic_growth_rate_010 = 0.10

crescimento_estimado010 = 2019.1710 + 0.0003 * optimistic_growth_rate_010
print(crescimento_estimado010)

# Cenário pessimista: crescimento de 5% ao ano
pessimistic_growth_rate_05 = 0.05

crescimento_estimado05 = 2019.1710 + 0.0003 * pessimistic_growth_rate_05
print(crescimento_estimado05)


# Cenário realista: crescimento de 7% ao ano
realistic_growth_rate = 0.07

crescimento_estimado07 = 2019.1710 + 0.0003 * realistic_growth_rate
print(crescimento_estimado07)


modelo_ano = pd.DataFrame({'ModelYear':[10]})
quantidade_estimada = lm.predict(crescimento_estimado07)[0]
print(quantidade_estimada)